In [1]:
import logging
import os
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.PcitExperiment import PcitExperiment
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver

pd.options.display.max_columns = 999

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=2,
    seed=0,
)

In [4]:
t_c = 300
t_v = 300
K = 2
n = 1
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = GlobalExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-14 18:29:16] INFO      [GLOBAL] Start!
[2024-11-14 18:29:17] INFO      Attempt 1/1
[2024-11-14 18:29:17] DEBUG     SMAC configuration, time: [300. 300.]
[2024-11-14 18:29:17] DEBUG     executor start
[2024-11-14 18:29:17] DEBUG     (0, 0) fn submitted
[2024-11-14 18:29:17] DEBUG     (0, 1) fn submitted
[2024-11-14 18:29:17] DEBUG     (1, 0) fn submitted
[2024-11-14 18:29:17] DEBUG     (1, 1) fn submitted
[2024-11-14 18:29:17] DEBUG     (0, 0) result
[2024-11-14 18:29:21] DEBUG     (0, 0) result inserted
[2024-11-14 18:29:21] DEBUG     (0, 1) result
[2024-11-14 18:29:21] DEBUG     (0, 1) result inserted
[2024-11-14 18:29:21] DEBUG     (1, 0) result
[2024-11-14 18:29:21] DEBUG     (1, 0) result inserted
[2024-11-14 18:29:21] DEBUG     (1, 1) result
[2024-11-14 18:29:21] DEBUG     (1, 1) result inserted
[2024-11-14 18:29:21] DEBUG     executor shutdown
[2024-11-14 18:29:21] DEBUG     SMAC iteration 1, cost: 1.93, configuration time: [296.08 296.12]
[2024-11-14 18:29:23] DEBUG    

In [4]:
t_c = 100
t_v = 100
K = 2
n = 1
max_iter = 1
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = PcitExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-13 23:13:39] INFO      [PCIT] Start!
[2024-11-13 23:13:39] DEBUG     executor start
[2024-11-13 23:13:46] DEBUG     data/TSP/CEPS_benchmark/cluster_netgen/40.tsp features calculated
[2024-11-13 23:13:46] DEBUG     data/TSP/CEPS_benchmark/grid/04.tsp features calculated
[2024-11-13 23:13:46] DEBUG     (0, 0) fn submitted
[2024-11-13 23:13:46] DEBUG     (0, 1) fn submitted
[2024-11-13 23:13:46] DEBUG     (1, 0) fn submitted
[2024-11-13 23:13:46] DEBUG     (1, 1) fn submitted
[2024-11-13 23:13:46] DEBUG     (0, 0) result
[2024-11-13 23:13:59] ERROR     timeout: instance data/TSP/CEPS_benchmark/cluster_netgen/40.tsp, solver 3401794975394163415
[2024-11-13 23:13:59] DEBUG     (0, 0) result inserted
[2024-11-13 23:13:59] DEBUG     (0, 1) result
[2024-11-13 23:14:12] ERROR     timeout: instance data/TSP/CEPS_benchmark/cluster_netgen/40.tsp, solver -1048468781324309231
[2024-11-13 23:14:12] DEBUG     (0, 1) result inserted
[2024-11-13 23:14:12] DEBUG     (1, 0) result
[2024-11-13 23:1

In [4]:
t_c = 30
t_v = 30
K = 2
n = 2
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance

experiment = ParhydraExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-13 23:18:27] INFO      [PARHYDRA] Start!
[2024-11-13 23:18:27] INFO      Solver 1/2
[2024-11-13 23:18:27] INFO      Attempt 1/2
[2024-11-13 23:18:27] DEBUG     SMAC configuration, time: [30.]
[2024-11-13 23:18:27] DEBUG     executor start
[2024-11-13 23:18:27] DEBUG     (0, 0) fn submitted
[2024-11-13 23:18:27] DEBUG     (1, 0) fn submitted
[2024-11-13 23:18:27] DEBUG     (0, 0) result
[2024-11-13 23:18:31] DEBUG     (0, 0) result inserted
[2024-11-13 23:18:31] DEBUG     (1, 0) result
[2024-11-13 23:18:31] DEBUG     (1, 0) result inserted
[2024-11-13 23:18:31] DEBUG     executor shutdown
[2024-11-13 23:18:31] DEBUG     SMAC iteration 1, cost: 1.77, configuration time: [26.46]
[2024-11-13 23:18:31] DEBUG     executor start
[2024-11-13 23:18:31] DEBUG     (0, 0) cached result
[2024-11-13 23:18:31] DEBUG     (1, 0) cached result
[2024-11-13 23:18:31] DEBUG     (0, 0) future None
[2024-11-13 23:18:31] DEBUG     (1, 0) future None
[2024-11-13 23:18:31] DEBUG     executor shutdown
[

In [ ]:
remaining_time = np.ones(shape=(K, )) * np.inf
best_portfolio.evaluate(test_instances, remaining_time, "test")

In [7]:
from src.database import db_connect
conn = db_connect()

In [ ]:
import sqlite3
conn = sqlite3.connect("database/2024_11_12_23_10_13.db")

In [8]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,50,0,NO,ALPHA,0,QUADRANT,YES,YES,1,WALK,1.000000,1,0,2147483647,5,5,1,0,50,YES,YES,0,YES
1,RYHPRufvqQEhc1-aiGnUaccMdjqBNa6UAi9mFPBhyFk=,45,1,YES,ALPHA,9,QUADRANT,YES,YES,2,QUICK-BORUVKA,0.562941,0,0,1057551428,8,2,0,2,70,YES,NO,6,NO
2,Lk9sIgdjzFhbnR3nwXz06LS79w-ggwn54qLy5zOUFlQ=,58,1,YES,NEAREST-NEIGHBOR,6,QUADRANT,NO,NO,4,WALK,0.494674,5,0,1428567367,6,3,5,5,55,NO,NO,2,YES
3,HQ-k4vHEFNqdO_auCkYeFzYu44p6u93NLkneiN-caKg=,56,0,YES,NEAREST-NEIGHBOR,6,QUADRANT,YES,YES,4,GREEDY,0.338775,1,5,1200129777,5,3,0,4,99,YES,YES,3,YES
4,eRi4rsLLAgHfCLpKEXyw3xB3-qECmeMsmP9bdM46b3M=,59,1,NO,NEAREST-NEIGHBOR,10,QUADRANT,NO,NO,4,SIERPINSKI,0.912056,3,0,1196232513,1,3,4,5,2,NO,YES,3,YES
5,ohTZR_X4TPHI0MIXzbxR6lqqJ8djfZuzY_PhdR0hT9Q=,48,0,NO,DELAUNAY,9,QUADRANT,NO,YES,2,GREEDY,0.126676,2,0,750792518,2,3,2,4,25,NO,YES,6,YES
6,E2Rak611YIonr5igC6FMkdy7gfi4hx0GvSPuW_tkwL4=,45,1,NO,QUADRANT,2,QUADRANT,YES,YES,1,SIERPINSKI,0.585623,5,4,1666648710,8,5,5,0,32,NO,YES,2,YES
7,KtfFnpDYFRWrg9g37O-yliY70_3s2IHWjOKrjLm7faw=,58,0,NO,QUADRANT,2,QUADRANT,YES,YES,2,GREEDY,0.570384,4,0,714281479,10,4,4,3,6,NO,NO,0,NO
8,rtlyglyEgp4PVisqE9z0OVt3isX5RrfomZfS219fkUg=,56,1,YES,DELAUNAY,3,QUADRANT,YES,YES,1,SIERPINSKI,0.045117,0,5,1450992209,10,2,4,0,75,YES,NO,0,YES
9,M0V6m9fW0DOCrqiHuaW5IkJ2c7fj_589A1A7m9uD5xo=,45,0,NO,NEAREST-NEIGHBOR,3,QUADRANT,YES,YES,1,GREEDY,0.438081,0,0,1804794397,9,5,1,2,22,NO,YES,3,YES


In [9]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp
1,data/TSP/CEPS_benchmark/grid/04.tsp


In [10]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.61,2.61,configuration,2024-11-14 17:29:21
1,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.55,2.55,configuration,2024-11-14 17:29:21
2,data/TSP/CEPS_benchmark/grid/04.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.31,1.31,configuration,2024-11-14 17:29:21
3,data/TSP/CEPS_benchmark/grid/04.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.33,1.33,configuration,2024-11-14 17:29:21
4,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,RYHPRufvqQEhc1-aiGnUaccMdjqBNa6UAi9mFPBhyFk=,100.00,10.00,configuration,2024-11-14 17:29:35
...,...,...,...,...,...,...
61,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,105J0VKTkUKnq5-5hnPpwaAFZTXN7hpOYTKnukhY5qI=,100.00,10.00,configuration,2024-11-14 17:33:09
62,data/TSP/CEPS_benchmark/grid/04.tsp,17xqXH2-ZS4eS1aVQ5jW8WdkkVQqaaKRyWb8FW2nFE4=,100.00,10.00,configuration,2024-11-14 17:33:10
63,data/TSP/CEPS_benchmark/grid/04.tsp,105J0VKTkUKnq5-5hnPpwaAFZTXN7hpOYTKnukhY5qI=,100.00,10.00,configuration,2024-11-14 17:33:10
64,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6rDEaVnSm_0HFwUlv2U3gmljZaT27QGWtkQTpewLMT4=,100.00,10.00,configuration,2024-11-14 17:33:21


In [11]:
df.loc[df["comment"] == "configuration"]

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.61,2.61,configuration,2024-11-14 17:29:21
1,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.55,2.55,configuration,2024-11-14 17:29:21
2,data/TSP/CEPS_benchmark/grid/04.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.31,1.31,configuration,2024-11-14 17:29:21
3,data/TSP/CEPS_benchmark/grid/04.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.33,1.33,configuration,2024-11-14 17:29:21
4,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,RYHPRufvqQEhc1-aiGnUaccMdjqBNa6UAi9mFPBhyFk=,100.00,10.00,configuration,2024-11-14 17:29:35
...,...,...,...,...,...,...
61,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,105J0VKTkUKnq5-5hnPpwaAFZTXN7hpOYTKnukhY5qI=,100.00,10.00,configuration,2024-11-14 17:33:09
62,data/TSP/CEPS_benchmark/grid/04.tsp,17xqXH2-ZS4eS1aVQ5jW8WdkkVQqaaKRyWb8FW2nFE4=,100.00,10.00,configuration,2024-11-14 17:33:10
63,data/TSP/CEPS_benchmark/grid/04.tsp,105J0VKTkUKnq5-5hnPpwaAFZTXN7hpOYTKnukhY5qI=,100.00,10.00,configuration,2024-11-14 17:33:10
64,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6rDEaVnSm_0HFwUlv2U3gmljZaT27QGWtkQTpewLMT4=,100.00,10.00,configuration,2024-11-14 17:33:21


In [ ]:
_ = df.loc[df["comment"] == "test"].pivot_table(index="instance_id", columns="solver_id", values="cost")
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

In [ ]:
_["min"].describe()

In [ ]:
(_["min"] > 90).value_counts()

In [ ]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

In [ ]:
_.sort_values(by="min")

In [ ]:
config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
solver = TSP_LKH_Solver(config=config)

instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index